In [1]:
!pip install seqeval==1.0.0
!pip install konlpy
!bash < (curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
/usr/bin/sh: 1: Syntax error: "(" unexpected


## PosTag Model
- 4그램 단위로 가젯 벡터를 매겨봄. 
- 임베딩에 7차원 벡터를 추가.

In [2]:
'''
의존성 주입 헬퍼
'''
from settings import postag_model_config as config

def module_and_class(string):
    tok = string.split('.')
    mod = ".".join(tok[0:-1])
    cls = tok[-1]
    return __import__(mod, fromlist=[cls]), cls

def instantiate(string):
    mod, cls = module_and_class(string)
    return getattr(mod, cls)()

mecab = instantiate(config['tagger'])
pt = instantiate(config['features'][0])
pt.set_tagger(mecab)

In [3]:
import os, sys
import torch
import torch.nn as nn
from torchcrf import CRF
from seqeval.metrics import classification_report

class RNN_CRF(nn.Module):
    def __init__(self, config):
        super(RNN_CRF, self).__init__()

        # 전체 음절 개수
        self.eumjeol_vocab_size = config["word_vocab_size"]

        # 음절 임베딩 사이즈
        self.embedding_size = config["embedding_size"]

        # GRU 히든 사이즈
        self.hidden_size = config["hidden_size"]

        # 분류할 태그의 개수
        self.number_of_tags = config["number_of_tags"]
        
        self.gru_input_size = self.embedding_size + config["postag_feature_length"]

        # 입력 데이터에 있는 각 음절 index를 대응하는 임베딩 벡터로 치환해주기 위한 임베딩 객체
        self.embedding = nn.Embedding(num_embeddings=self.eumjeol_vocab_size,
                                      embedding_dim=self.embedding_size,
                                      padding_idx=0)
        
        self.dropout = nn.Dropout(config["dropout"])

        # Bi-GRU layer
        self.bi_gru = nn.GRU(input_size=self.gru_input_size,
                             hidden_size= self.hidden_size,
                             num_layers=1,
                             batch_first=True,
                             bidirectional=True)

        # CRF layer
        self.crf = CRF(num_tags=self.number_of_tags, batch_first=True)

        # fully_connected layer를 통하여 출력 크기를 number_of_tags에 맞춰줌
        # (batch_size, max_length, hidden_size*2) -> (batch_size, max_length, number_of_tags)
        self.hidden2num_tag = nn.Linear(in_features=self.hidden_size*2, out_features=self.number_of_tags)

    def forward(self, inputs, postags, labels=None):
        # (batch_size, max_length) -> (batch_size, max_length, embedding_size)
        eumjeol_inputs = self.embedding(inputs)
        eumjeol_gazette_inputs = torch.cat((eumjeol_inputs, postags), -1)
        encoder_outputs, hidden_states = self.bi_gru(eumjeol_gazette_inputs)
        
        # (batch_size, curr_max_length, hidden_size*2)
        d_hidden_outputs = self.dropout(encoder_outputs)

        # (batch_size, curr_max_length, hidden_size*2) -> (batch_size, curr_max_length, number_of_tags)
        logits = self.hidden2num_tag(d_hidden_outputs)

        if(labels is not None):
            log_likelihood = self.crf(emissions=logits,
                                      tags=labels,
                                      reduction="mean")

            loss = log_likelihood * -1.0

            return loss
        else:
            output = self.crf.decode(emissions=logits)

            return output

In [4]:
from tqdm import tqdm
import numpy as np

def load_vocab(f_name):
    vocab_file = open(os.path.join(config['root_dir'], f_name),'r',encoding='utf8')
    print("{} vocab file loading...".format(f_name))

    # default 요소가 저장된 딕셔너리 생성
    symbol2idx, idx2symbol = {"<PAD>":0, "<UNK>":1}, {0:"<PAD>", 1:"<UNK>"}

    # 시작 인덱스 번호 저장
    index = len(symbol2idx)
    for line in tqdm(vocab_file.readlines()):
        symbol = line.strip()
        symbol2idx[symbol] = index
        idx2symbol[index]= symbol
        index+=1

    return symbol2idx, idx2symbol

def convert_data2feature(data, symbol2idx, max_length=None):
    feature = np.zeros(shape=(max_length), dtype=np.int)
    words = data.split()

    for idx, word in enumerate(words[:max_length]):
        if word in symbol2idx.keys():
            feature[idx] = symbol2idx[word]
        else:
            feature[idx] = symbol2idx["<UNK>"]
    return feature

# 파라미터로 입력받은 파일로부터 tensor객체 생성
def load_data(config, f_name, word2idx, tag2idx):
    file = open(os.path.join(config['root_dir'], f_name),'r',encoding='utf8')

    # return할 문장/라벨 리스트 생성
    indexing_inputs, indexing_tags = [], []
    postags = []

    print("{} file loading...".format(f_name))

    # 실제 데이터는 아래와 같은 형태를 가짐
    # 문장 \t 태그
    # 세 종 대 왕 은 <SP> 조 선 의 <SP> 4 대 <SP> 왕 이 야 \t B_PS I_PS I_PS I_PS O <SP> B_LC I_LC O <SP> O O <SP> O O O
    for line in tqdm(file.readlines()):
        try:
            id, sentence, tags = line.strip().split('\t')
        except:
            id, sentence = line.strip().split('\t')
        input_sentence = convert_data2feature(sentence, word2idx, config["max_length"])
        input_postag = pt.transform_end2end(sentence, config["max_length"])
        indexing_tag = convert_data2feature(tags, tag2idx, config["max_length"])

        indexing_inputs.append(input_sentence)
        postags.append(input_postag)
        indexing_tags.append(indexing_tag)
        
    indexing_inputs = torch.tensor(indexing_inputs, dtype=torch.long)
    postags = torch.tensor(postags, dtype=torch.float)
    indexing_tags = torch.tensor(indexing_tags, dtype=torch.long)

    return indexing_inputs, postags, indexing_tags

# tensor 객체를 리스트 형으로 바꾸기 위한 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

In [5]:
from torch.utils.data import (DataLoader, TensorDataset)
import torch.optim as optim

def train(config):
    # 모델 객체 생성
    model = RNN_CRF(config).cuda()
    # 단어 딕셔너리 생성
    word2idx, idx2word = load_vocab(config["word_vocab_file"])
    tag2idx, idx2tag = load_vocab(config["tag_vocab_file"])

    # 데이터 Load
    train_input_features, train_postags, train_tags = load_data(config, config["train_file"], word2idx, tag2idx)
    test_input_features, test_postags, test_tags = load_data(config, config["dev_file"], word2idx, tag2idx)

    # 불러온 데이터를 TensorDataset 객체로 변환
    train_features = TensorDataset(train_input_features, train_postags, train_tags)
    train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

    test_features = TensorDataset(test_input_features, test_postags, test_tags)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=config["batch_size"])

    # 모델을 학습하기위한 optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    accuracy_list = []
    for epoch in range(config["epoch"]):
        model.train()
        losses = []
        for step, batch in enumerate(train_dataloader):
            # .cuda()를 이용하여 메모리에 업로드
            batch = tuple(t.cuda() for t in batch)
            input_features, postags, labels = batch

            # loss 계산
            loss = model.forward(input_features, postags, labels)

            # 변화도 초기화
            optimizer.zero_grad()

            # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
            loss.backward()

            # 모델 내부 각 매개변수 가중치 갱신
            optimizer.step()

            if (step + 1) % 50 == 0:
                print("{} step processed.. current loss : {}".format(step + 1, loss.data.item()))
            losses.append(loss.data.item())



        print("Average Loss : {}".format(np.mean(losses)))

        # 모델 저장
        torch.save(model.state_dict(), os.path.join(config["output_dir_path"], "epoch_{}.pt".format(epoch + 1)))

        do_test(model, test_dataloader, idx2tag)



def test(config):
    # 모델 객체 생성
    model = RNN_CRF(config).cuda()
    # 단어 딕셔너리 생성
    word2idx, idx2word = load_vocab(config["word_vocab_file"])
    tag2idx, idx2tag = load_vocab(config["tag_vocab_file"])


    # 저장된 가중치 Load
    model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["trained_model_name"])))

    # 데이터 Load
    test_input_features, test_tags = load_data(config, config["dev_file"], word2idx, tag2idx)

    # 불러온 데이터를 TensorDataset 객체로 변환
    test_features = TensorDataset(test_input_features, test_tags)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=config["batch_size"])
    # 평가 함수 호출
    do_test(model, test_dataloader, idx2tag)

def do_test(model, test_dataloader, idx2tag):
    model.eval()
    predicts, answers = [], []
    for step, batch in enumerate(test_dataloader):
        # .cuda() 함수를 이용하요 메모리에 업로드
        batch = tuple(t.cuda() for t in batch)

        # 데이터를 각 변수에 저장
        input_features, postags, labels = batch

        # 예측 라벨 출력
        output = model(input_features, postags)

        # 성능 평가를 위해 예측 값과 정답 값 리스트에 저장
        for idx, answer in enumerate(tensor2list(labels)):
            answers.extend([idx2tag[e].replace("_", "-") for e in answer if idx2tag[e] != "<SP>" and idx2tag[e] != "<PAD>"])
            predicts.extend([idx2tag[e].replace("_", "-") for i, e in enumerate(output[idx]) if idx2tag[answer[i]] != "<SP>" and idx2tag[answer[i]] != "<PAD>"] )
    
    # 성능 평가
    print(classification_report(answers, predicts))


In [6]:
##########################################################
#                                                        #
#        평가 기준이 되는 지표는 Macro F1 Score          #
#           제출 포맷은 id \t predict_tag                #
#            25 \t B_PS I_PS <SP> O O O ...              #
#                                                        #
##########################################################


import os
if(__name__=="__main__"):
    output_dir = os.path.join(config['root_dir'], "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)


  7%|▋         | 479/7319 [00:00<00:01, 4785.19it/s]

vocab/word_vocab.txt vocab file loading...
vocab/tag_vocab.txt vocab file loading...
ner_train.txt file loading...


 25%|██▌       | 252/995 [00:00<00:00, 2518.38it/s]

ner_dev.txt file loading...


100%|██████████| 995/995 [00:00<00:00, 3505.88it/s]


...

              precision    recall  f1-score   support

          DT       0.57      0.52      0.55       622
          LC       0.66      0.34      0.45       535
          OG       0.46      0.21      0.29       971
          PS       0.46      0.40      0.43       739
          TI       0.29      0.23      0.26        95

   micro avg       0.51      0.35      0.41      2962
   macro avg       0.49      0.34      0.39      2962
weighted avg       0.52      0.35      0.41      2962

50 step processed.. current loss : 10.767030715942383
100 step processed.. current loss : 9.746017456054688
Average Loss : 11.706230868463932
              precision    recall  f1-score   support

          DT       0.74      0.69      0.72       622
          LC       0.59      0.58      0.58       535
          OG       0.66      0.38      0.48       971
          PS       0.63      0.55      0.59       739
          TI       0.78      0.64      0.71        95

   micro avg       0.65      0.53      0.59

50 step processed.. current loss : 1.5862340927124023
100 step processed.. current loss : 2.060638427734375
Average Loss : 1.3965333948964658
              precision    recall  f1-score   support

          DT       0.76      0.73      0.74       622
          LC       0.71      0.65      0.68       535
          OG       0.65      0.62      0.64       971
          PS       0.72      0.65      0.68       739
          TI       0.79      0.66      0.72        95

   micro avg       0.71      0.66      0.68      2962
   macro avg       0.73      0.66      0.69      2962
weighted avg       0.71      0.66      0.68      2962

50 step processed.. current loss : 2.0064945220947266
100 step processed.. current loss : 1.7111234664916992
Average Loss : 1.2706528363020524
              precision    recall  f1-score   support

          DT       0.72      0.73      0.73       622
          LC       0.65      0.66      0.65       535
          OG       0.63      0.61      0.62       971
         

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:45: UserWarning: <SP> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

          DT       0.75      0.74      0.74       622
          LC       0.65      0.69      0.67       535
          OG       0.66      0.59      0.62       971
          PS       0.65      0.66      0.65       739
          TI       0.73      0.68      0.71        95

   micro avg       0.68      0.66      0.67      2962
   macro avg       0.69      0.67      0.68      2962
weighted avg       0.68      0.66      0.67      2962

50 step processed.. current loss : 0.5981247425079346
100 step processed.. current loss : 1.1348077058792114
Average Loss : 0.964946718060452
              precision    recall  f1-score   support

          DT       0.79      0.74      0.76       622
          LC       0.71      0.60      0.65       535
          OG       0.69      0.55      0.61       971
          PS       0.69      0.62      0.65       739
          TI       0.75      0.73      0.74        95

   micro avg       0.72      0.62      0.67

              precision    recall  f1-score   support

          DT       0.77      0.73      0.75       622
          LC       0.73      0.64      0.68       535
          OG       0.67      0.57      0.61       971
          PS       0.73      0.61      0.67       739
          TI       0.70      0.68      0.69        95

   micro avg       0.72      0.63      0.67      2962
   macro avg       0.72      0.65      0.68      2962
weighted avg       0.72      0.63      0.67      2962

50 step processed.. current loss : 0.7375630736351013
100 step processed.. current loss : 1.1635046005249023
Average Loss : 0.8654801519020744
              precision    recall  f1-score   support

          DT       0.77      0.72      0.75       622
          LC       0.67      0.64      0.65       535
          OG       0.67      0.55      0.60       971
          PS       0.68      0.65      0.67       739
          TI       0.69      0.66      0.68        95

   micro avg       0.69      0.63      0.6

In [7]:
print(pt.label2idx)

{'UNK': 0, 'NNG': 1, 'NNP': 2, 'NNB': 3, 'NNBC': 4, 'NR': 5, 'NP': 6, 'VV': 7, 'VA': 8, 'VX': 9, 'VCP': 10, 'VCN': 11, 'MM': 12, 'MAG': 13, 'MAJ': 14, 'IC': 15, 'JKS': 16, 'JKC': 17, 'JKG': 18, 'JKO': 19, 'JKB': 20, 'JKV': 21, 'JKQ': 22, 'JX': 23, 'JC': 24, 'EP': 25, 'EF': 26, 'EC': 27, 'ETN': 28, 'ETM': 29, 'XPN': 30, 'XSN': 31, 'XSV': 32, 'XSA': 33, 'XR': 34, 'SF': 35, 'SE': 36, 'SSO': 37, 'SSC': 38, 'SC': 39, 'SY': 40, 'SL': 41, 'SH': 42, 'SN': 43}
